# Setup

Make sure you use Kaggle's P100 GPU. This notebook has not been tested with any other GPU.

In [40]:
import os
import wandb
from kaggle_secrets import UserSecretsClient

try:
    user_secrets = UserSecretsClient()
    wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")
    os.environ["WANDB_API_KEY"] = wandb_api_key
    wandb.login(key=wandb_api_key)
except Exception as e:
    print("WANDB_API_KEY not set or failed to load.")
    print("Reason:", str(e))
    print("In Kaggle, add it via Add-ons → Secrets → Add Secret.")

MODEL_UIDS = [
    "unsloth/llama-3.2-1B-bnb-4bit",
    "unsloth/llama-3.2-3B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
]
DATASET_SFT_UIDS = [
    "allenai/tulu-3-sft-personas-math-grade",
    "allenai/tulu-3-sft-personas-math",
    "allenai/tulu-3-sft-personas-instruction-following",
    "allenai/tulu-3-sft-personas-algebra",
    "allenai/tulu-3-sft-personas-code",
]
DATASET_SFT_MIXTURE_UIDS = ["allenai/tulu-3-sft-mixture"]
LM_EVAL_UIDS = [
    "hellaswag",
    "gsm8k",
    "arc_easy",
    "truthfulqa",
    "winogrande",
    "humaneval",
]

MODEL_UID = MODEL_UIDS[0]  # 0 is 1B, 1 is 3B, 2 is 8B
DATASET_UIDS = DATASET_SFT_UIDS # make sure this represents the datasets you're currently interested in
DATASET_UID = DATASET_UIDS[3] # choose your dataset

MAX_STEPS = 1  # edit this after debugging
LIMIT = 1 # edit this after debugging

print("Config done...")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Config done...


In [5]:
!git clone https://github.com/EleutherAI/lm-evaluation-harness.git
!pip install -e ./lm-evaluation-harness/.
!pip install unsloth transformers datasets wandb pandas

fatal: destination path 'lm-evaluation-harness' already exists and is not an empty directory.
Obtaining file:///kaggle/working/lm-evaluation-harness
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━

# Fine-Tuning

In [6]:
from unsloth import FastLanguageModel
import wandb
from transformers import BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig
import torch
from datasets import load_dataset
import json

model_name = MODEL_UID.split("/")[-1]
dataset_name = DATASET_UID.split("/")[-1]

base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_UID, max_seq_length=2048, dtype=None, load_in_4bit=True
)

base_model.save_pretrained(f"{model_name}")
tokenizer.save_pretrained(f"{model_name}")

model = FastLanguageModel.get_peft_model(
    base_model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    # random_state = 3407,
    max_seq_length=2048,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

model.config.quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

train_dataset = load_dataset(DATASET_UID, split="train")
print(train_dataset[0].keys())
print(json.dumps(train_dataset[0]["messages"], indent=2))


def formatting_func(examples):
    messages = examples["messages"]
    texts = [
        "".join([m["content"].strip() + "\n" for m in convo]).strip()
        for convo in messages
    ]
    return {"text": texts}


train_dataset = train_dataset.map(formatting_func, batched=True)

wandb.login(key=os.environ["WANDB_API_KEY"])
wandb.init(
    project="pm-pt",
    name=f"{model_name}_{dataset_name}",
    config={
        "model": MODEL_UID,
        "dataset": DATASET_UID,
        "max_steps": MAX_STEPS,
        "learning_rate": 2e-4,
        "batch_size": 2,
        "gradient_accumulation_steps": 4,
    },
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    args=SFTConfig(
        dataset_text_field="text",
        max_seq_length=2048,
        learning_rate=2e-4,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=MAX_STEPS,
        report_to="wandb",
        run_name=f"{model_name}_{dataset_name}",
        output_dir="outputs",
        optim="adamw_8bit",
    ),
)

trainer.train()

wandb.finish()

model.save_pretrained(f"{model_name}_finetuned_{dataset_name}")
tokenizer.save_pretrained(f"{model_name}_finetuned_{dataset_name}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-04-30 09:18:27.736298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746004707.914467      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746004707.970988      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.3: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 1 | Total steps = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.996700


train/epoch,▁▁
train/global_step,▁▁
train/grad_norm,▁
train/learning_rate,▁
train/loss,▁
total_flos,50099016327168.0
train/epoch,0.0004
train/global_step,1
train/grad_norm,0.22184
train/learning_rate,0
train/loss,0.9967


('llama-3.2-1B-bnb-4bit_finetuned_tulu-3-sft-personas-algebra/tokenizer_config.json',
 'llama-3.2-1B-bnb-4bit_finetuned_tulu-3-sft-personas-algebra/special_tokens_map.json',
 'llama-3.2-1B-bnb-4bit_finetuned_tulu-3-sft-personas-algebra/tokenizer.json')

# Evaluation

In [70]:
import os
import subprocess

model_name = MODEL_UID.split("/")[-1]
dataset_name = DATASET_UID.split("/")[-1]

peft_path = f"./{model_name}_finetuned_{dataset_name}"

os.environ["HF_ALLOW_CODE_EVAL"] = "1"

tasks_str = ",".join(LM_EVAL_UIDS)

command = [
    "lm_eval",
    "--model",
    "hf",
    "--model_args",
    f"pretrained=./{model_name},peft={peft_path}",
    "--tasks",
    tasks_str,
    "--confirm_run_unsafe_code",
    "--device",
    "cuda",
    "--batch_size",
    "auto",
    "--limit",
    str(LIMIT),
]

subprocess.run(command)

2025-04-30 10:18:47.586715: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746008327.609704    1839 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746008327.616519    1839 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Running generate_until requests:   0%|          | 0/3 [00:00<?, ?it/s]

Passed argument batch_size = auto:1. Detecting largest batch size
Determined largest batch size: 64


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Running generate_until requests: 100%|██████████| 3/3 [01:10<00:00, 23.43s/it]


Passed argument batch_size = auto. Detecting largest batch size
Determined Largest batch size: 1


fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


hf (pretrained=./llama-3.2-1B-bnb-4bit,peft=./llama-3.2-1B-bnb-4bit_finetuned_tulu-3-sft-personas-algebra), gen_kwargs: (None), limit: 1.0, num_fewshot: None, batch_size: auto (64)
|    Tasks     |Version|     Filter     |n-shot|  Metric   |   |Value |   |Stderr|
|--------------|------:|----------------|-----:|-----------|---|-----:|---|------|
|arc_easy      |      1|none            |     0|acc        |↑  |0.0000|±  |   N/A|
|              |       |none            |     0|acc_norm   |↑  |1.0000|±  |   N/A|
|gsm8k         |      3|flexible-extract|     5|exact_match|↑  |0.0000|±  |   N/A|
|              |       |strict-match    |     5|exact_match|↑  |0.0000|±  |   N/A|
|hellaswag     |      1|none            |     0|acc        |↑  |0.0000|±  |   N/A|
|              |       |none            |     0|acc_norm   |↑  |1.0000|±  |   N/A|
|humaneval     |      1|create_test     |     0|pass@1     |   |0.0000|±  |   N/A|
|truthfulqa_gen|      3|none            |     0|bleu_acc   |↑  |0.0000|±

CompletedProcess(args=['lm_eval', '--model', 'hf', '--model_args', 'pretrained=./llama-3.2-1B-bnb-4bit,peft=./llama-3.2-1B-bnb-4bit_finetuned_tulu-3-sft-personas-algebra', '--tasks', 'hellaswag,gsm8k,arc_easy,truthfulqa,winogrande,humaneval', '--confirm_run_unsafe_code', '--device', 'cuda', '--batch_size', 'auto', '--limit', '1'], returncode=0)

In [71]:
from itertools import product

columns = ["model_uid", "dataset_uid"] + LM_EVAL_UIDS

model_dataset_pairs = list(product(MODEL_UIDS, DATASET_UIDS))

empty_eval_df = pd.DataFrame(columns=columns)

for model_uid, dataset_uid in model_dataset_pairs:
    row = {
        "model_uid": model_uid,
        "dataset_uid": dataset_uid,
    }
    for task in LM_EVAL_UIDS:
        row[task] = None
    empty_eval_df.loc[len(empty_eval_df)] = row

empty_eval_df.to_excel("empty_eval_results.xlsx", index=False)

print("Created empty eval_results.xlsx")

Created empty eval_results.xlsx
